In [1]:
#Use this if you don't have all the packages installed, this will add them.
import Pkg
Pkg.add(["IterTools", "SparseArrays", "COSMO", "JuMP"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using IterTools
using LinearAlgebra, Random, SparseArrays, COSMO, JuMP

# Code

## Pauli Strings

In [111]:
#Get all Pauli strings of weight W
function enumerate_paulistrings(one_paulis::Vector{Char}, n::Int, W::Int)::Vector{String}
    res = String[]
    #Get all lists of W distinct indices
    for indices = IterTools.subsets(1:n, W)
        pauli_char_array = repeat(['I'],n)
        #And all 3^W Pauli strings on those indices
        for pauli_type = Iterators.product(repeat([one_paulis], W)...)
            for (i,ci) = enumerate(indices)
                pauli_char_array[ci] = pauli_type[i]
            end
            push!(res, String(pauli_char_array))
        end
    end
    return res
end

@assert length(enumerate_paulistrings(['F','G','H','J','K'],20,3)) == (5^3) * (20*19*18)/(6)
@assert length(enumerate_paulistrings(['X','Y'],5,6)) == 0 #nothing of weight higher than system size

#Multiplies two strings, returning a new string and a phase.
#Currently only works for qubits with normal XYZ labels.
function multiply_paulistrings(strA::String, strB::String)::Tuple{String, Complex}
    @assert length(strA) == length(strB) "Paulis must be same length"
    res_char_array = repeat(['I'],length(strA))
    acc_phase = Complex(1) #1 + 0im
    for (i,(a,b)) = enumerate(zip(strA, strB))
        if a==b
            continue #I^2=X^2=Y^2=Z^2 = I
        elseif a=='I'
            res_char_array[i] = b
        elseif b=='I'
            res_char_array[i] = a
        elseif a=='X' && b=='Y'
            res_char_array[i] = 'Z'
            acc_phase *= 1im
        elseif a=='Y' && b=='Z'
            res_char_array[i] = 'X'
            acc_phase *= 1im
        elseif a=='Z' && b=='X'
            res_char_array[i] = 'Y'
            acc_phase *= 1im
        elseif a=='X' && b=='Z'
            res_char_array[i] = 'Y'
            acc_phase *= -1im
        elseif a=='Y' && b=='X'
            res_char_array[i] = 'Z'
            acc_phase *= -1im
        elseif a=='Z' && b=='Y'
            res_char_array[i] = 'X'
            acc_phase *= -1im
        else
            error("Bad characters "+a+" and "+b)
        end
    end
    return (String(res_char_array), acc_phase)
end

@assert multiply_paulistrings("IIX","YIX") == ("YII",1)
@assert multiply_paulistrings("IXXIII","YYIIII") == ("YZXIII",1im)

## Variable allocation

In [4]:
#Make a fresh batch of variables for a new pseudostate
function make_qubit_SDP_vars(model, q, n, d, k)
    @assert k <= 2*d "Observable order k must be at most 2 times pseudostate level d"
    @assert 2*d <= n+1 "Level d is higher than make sense for only n qudits; only need ceil(n/2)."

    #Our nontrivial one-local operators (for qubits, the three Paulis). Should be tomographically complete
    one_paulis = ['X','Y','Z']
    @assert length(one_paulis) == q^2 - 1 "Incorrect number of local operators"

    #List of lists of Pauli strings for the 'rows' and 'columns' of our pseudostate. The Wth element is
    # all the weight-W Pauli strings we need. Goes up to weight d.
    row_ops = Vector{String}[]
    #And one big list of all operators in the whole matrix, up to weight 2d.
    mat_ops = String[]

    #Get the operators for each weight. Also, the trivial "III" column.
    row_III = repeat("I",n)
    for W = 1:d
        push!(row_ops, enumerate_paulistrings(one_paulis, n, W))
    end
    row_ops_flat = collect(Iterators.flatten(row_ops))

    mat_ops = copy(row_ops_flat)
    for W = d+1:2d
        append!(mat_ops, enumerate_paulistrings(one_paulis, n, W))
    end
    
    #First allocate the variables, all real expectations in the range -1 to 1:
    @variable(model, -1 ≤ vars[mat_ops] ≤ 1)
    #Now vars["XII"] is the XII expectation value.
    
    return (row_ops_flat, row_III, vars)
end

#Given a variable in a model, get its Pauli string.
function pauli_from_var(var)
    return name(var)[6:end-1]
end

pauli_from_var (generic function with 1 method)

## Pseudostate constraint

In [5]:
function make_pseudostate_model(model, vars, row_ops_flat, row_III)
    #Build the matrix of these variables for the pseudostate.
    #Place them in a matrix, with complex coefficients. With the complex coefficients, these are
    #type GenericAffExpr{ComplexF64, VariableRef}.
    pseudo_len = length(row_ops_flat)
    pseudo_mat = zeros(GenericAffExpr{ComplexF64, VariableRef}, 1+pseudo_len, 1+pseudo_len)
    for i=0:pseudo_len, j=0:pseudo_len
        if i == 0
            if j == 0
                pseudo_mat[1+i, 1+j] += 1
            else
                jstr = row_ops_flat[j]
                jvar = vars[jstr]
                pseudo_mat[1+i, 1+j] += jvar
            end
        elseif j == 0
            istr = row_ops_flat[i]
            ivar = vars[istr]
            pseudo_mat[1+i, 1+j] += ivar
        else
            istr = row_ops_flat[i]
            jstr = row_ops_flat[j]
            ijstr, phase = multiply_paulistrings(istr, jstr)
            if ijstr == row_III
                ijvar = 1
            else
                ijvar = vars[ijstr]
            end
            pseudo_mat[1+i, 1+j] += phase * ijvar
        end
    end

    #Need it as a LinearAlgebra.Hermitian type for JuMP.
    pseudo_mat_herm = Hermitian(pseudo_mat)
    @assert pseudo_mat_herm == pseudo_mat "Pseudostate SDP wasn't Hermitian"

    #Add constraint to the model.
    @constraint(model, pseudo_mat_herm in HermitianPSDCone())
end

function pseudostate_bound_expr(obj_expr, model = (0.0+1*obj_expr).terms.keys[1].model)
    #Get measurement to predict (lower bound):
    @objective(model, Min, obj_expr)
    JuMP.optimize!(model)
    if JuMP.primal_status(model) != FEASIBLE_POINT
        error("Infeasible input data, status = "*string(JuMP.termination_status(model)))
    end
    min_val = JuMP.value(obj_expr)
    println("Min value = ",min_val)

    @objective(model, Max, obj_expr)
    JuMP.optimize!(model)
    if JuMP.primal_status(model) != FEASIBLE_POINT
        error("Infeasible input data, status = "*string(JuMP.termination_status(model)))
    end
    max_val = JuMP.value(obj_expr)

    result_interval = [min_val, max_val]
    return result_interval
end

pseudostate_bound_expr (generic function with 2 methods)

## Unitary Evolution

In [10]:
#We describe a unitary as a sum of Paulis. In particular, it's an dictionary of terms, each is a Pauli string (of equal length)
# and a complex coefficient associated to it. Each string must be equal length, and it forms a unitary as long as the coefficients
#normalize to 1.
const Unitary = Dict{String, <:Complex}

#General observables are also sums of Paulis. Instead of requiring that the coefficients have norm 1, the coefficients must
#all be real.
const Observable = Dict{String, <:Real}

LoadError: invalid redefinition of constant Unitary

In [35]:
#Helper functions to get the d-locality of a unitary or an observable.
function locality(U::Unitary)
    return length(first(U)[1])
end

function locality(O::Observable)
    return length(first(O)[1])
end

function str_to_obs(S::String)::Observable
    return Dict{String,Real}(S => 1)
end

import Base.+, Base.*

function add!(O::Observable, change::Observable)
    for (s,w) = change
        if s ∉ keys(O)
            O[s] = w
        else
            O[s] += w
        end
    end
    return O
end

function +(O1::Observable, O2::Observable)
    return add!(copy(O1), O2)
end

function mul!(O::Observable, scl::Real)
    for (s,w) = O
        O[s] = w * scl
    end
    return O
end

function *(O::Observable, scl::Real)
    return mul!(copy(O), scl)
end

* (generic function with 402 methods)

In [88]:
#Given a unitary U and a Pauli string P, compute its conjugation as an observable.
function conjugate_pauli(U::Unitary, P::String)
    res = Dict{String, Real}()
    #(Each w1*s1 in U)† * (P) * (Each w2*s2 in U)
    for (s1,w1) = U
        w1 = conj(w1)
        s1P, w1P = multiply_paulistrings(s1, P)
        for (s2,w2) = U
            s1P2, w1P2 = multiply_paulistrings(s1P, s2)
            phase = w1 * w1P * w1P2 * w2
            coef = real(phase)
            if coef == 0.0
                continue
            end
            if s1P2 ∉ keys(res)
                res[s1P2] = 0.0
            end
            res[s1P2] += coef
        end
    end
    return res
end

function conjugate_observable(U::Unitary, O::Observable)
    res = Dict{String,Real}()
    for (k,v) = O
        UkU = conjugate_pauli(U, k)
        mul!(UkU, v)
        add!(res, UkU)
    end
    return filter(kv -> kv[2] != 0, res)
end

conjugate_observable (generic function with 1 method)

In [96]:
#Test: conjugate Z by a Hadamard.
H = Dict{String,Complex}("Z" => 1/sqrt(2), "X" => 1/sqrt(2))

println(conjugate_pauli(H, "Z"))

#Conjugate 0.5*ZI + 500*IY by a CZ.
CZ = Dict{String,Complex}("II" => 1/2, "IZ" => 1/2, "ZI" => 1/2, "ZZ" => -1/2)
obs = str_to_obs("ZI")*0.5 + str_to_obs("IY")*500
obs = conjugate_observable(CZ, obs)
println(obs)

#Further conjugate by a CNOT (which gives the original value again)
CNOT = Dict{String,Complex}("II" => 1/2, "IX" => 1/2, "ZI" => 1/2, "ZX" => -1/2)
obs = conjugate_observable(CNOT, obs)
println(obs)

Dict{String, Real}("Z" => 0.0, "X" => 0.9999999999999998)
Dict{String, Real}("ZI" => 0.5, "ZY" => 500.0)
Dict{String, Real}("ZI" => 0.5, "IY" => 500.0)


In [114]:
#Given a set of level-d variables, apply a 1-local unitary
#TODO

#Given a set of level-d variables, add new variables to make a level-(d+1) set
function extend_SDP_vars(model, d, old_vars)
    #Unregister the old "vars" in the model
    unregister(model, :vars)
    
    #Get the new variable names needed
    new_row_flat, row_III, new_vars = make_qubit_SDP_vars(model, q, n, d+1, k)

    #Make a new pseudostate matrix with them
    make_pseudostate_model(model, new_vars, new_row_flat, row_III)

    #Set them equal to the projections of the original ones
    for old_var = old_vars
        pauli_name = pauli_from_var(old_var)
        new_var = new_vars[pauli_name]
        @constraint(model, old_var == new_var)
    end
    return new_vars
end

#Given a level-(d+1) set of variables and a 2-local unitary, give the projection level-2 values.
function project_SDP_vars(model, d, lifted_vars, U::Unitary)
    #Unregister the old "vars" in the model
    unregister(model, :vars)
    
    #Get the new variable names needed
    new_row_flat, row_III, new_vars = make_qubit_SDP_vars(model, q, n, d, k)

    #Don't need to make a pseudostate constraint, this is automatically implied by the projection.
    # would be   make_pseudostate_model(model, new_vars, new_row_flat, row_III)
    
    #Set them equal to the lifts of the new observables (Heisenberg picture)
    for new_var = new_vars
        pauli_name = pauli_from_var(new_var)
        heisenberg_obs = conjugate_pauli(U, pauli_name)
        obs_expr = 0.0
        for (pauli_str, coef) = heisenberg_obs
            if pauli_str == repeat("I",n)
                obs_expr += coef
            else
                obs_expr += coef * lifted_vars[pauli_str]
            end
        end
        @constraint(model, new_var == obs_expr)
    end
    return new_vars
end
#TODO

project_SDP_vars (generic function with 2 methods)

# Experiments

## PAC learning

### 3 Qubits

In [6]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 3 #3 qubits
d = 1 #level-1 pseudostate
k = 2 #looking at observables of at most order 2
;

In [8]:
model = JuMP.Model(optimizer_with_attributes(COSMO.Optimizer, "verbose" => false));

(row_ops_flat, row_III, vars) = make_qubit_SDP_vars(model, q, n, d, k);

make_pseudostate_model(model, vars, row_ops_flat, row_III)

#PAC framework:
#Add in measurement data as constraints:
@constraint(model, 0.3 ≤ vars["IIX"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIY"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["XIY"] ≤ -0.38)
@constraint(model, 0.3 ≤ vars["YIZ"] ≤ 0.4)
@constraint(model, 0.2 ≤ vars["ZIX"] ≤ 0.25)
@constraint(model, 0.5 ≤ vars["IYZ"] ≤ 0.55)
@constraint(model, 0.72 ≤ vars["XZI"] ≤ 0.74)
@constraint(model, 0.72 ≤ vars["YXI"] ≤ 0.74)

#The "@time" prints out the time taken for this computation
@time res_interval = pseudostate_bound_expr(vars["IIX"])
println("Possible interval for IIX: ",res_interval)

#The majority of the solve time is the first minimization. The second solve, the maximization, is usually quite a bit faster.
println("Time for maximization solve was ",solve_time(model))

@time res_interval = pseudostate_bound_expr(vars["YIZ"])
println("Possible interval YIZ: ",res_interval)

Min value = 0.30000000000000004
  0.056490 seconds (19.82 k allocations: 7.695 MiB)
Possible interval for IIX: [0.30000000000000004, 0.35480327522055477]
Time for maximization solve was 0.03197288513183594
Min value = 0.29999999999993454
  0.046009 seconds (13.66 k allocations: 6.408 MiB)
Possible interval YIZ: [0.29999999999993454, 0.3999999999999706]


Rerunning the same data at pseudostate level 2 now:

In [9]:
d = 2 #change d=1 to d=2

model = JuMP.Model(optimizer_with_attributes(COSMO.Optimizer, "verbose" => false));
(row_ops_flat, row_III, vars) = make_qubit_SDP_vars(model, q, n, d, k);
make_pseudostate_model(model, vars, row_ops_flat, row_III)

#PAC framework:
#Add in measurement data as constraints:
@constraint(model, 0.3 ≤ vars["IIX"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIY"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["XIY"] ≤ -0.38)
@constraint(model, 0.3 ≤ vars["YIZ"] ≤ 0.4)
@constraint(model, 0.2 ≤ vars["ZIX"] ≤ 0.25)
@constraint(model, 0.5 ≤ vars["IYZ"] ≤ 0.55)
@constraint(model, 0.72 ≤ vars["XZI"] ≤ 0.74)
@constraint(model, 0.72 ≤ vars["YXI"] ≤ 0.74)

@time res_interval = pseudostate_bound_expr(vars["IIX"])
println("Possible interval for IIX: ",res_interval)
println("Time for maximization solve was ",solve_time(model))

@time res_interval = pseudostate_bound_expr(vars["YIZ"])
println("Possible interval YIZ: ",res_interval)

Min value = 0.30000000000000004
  1.942969 seconds (117.18 k allocations: 91.951 MiB, 1.54% gc time, 1.94% compilation time)
Possible interval for IIX: [0.30000000000000004, 0.3034448202654543]
Time for maximization solve was 0.513416051864624
Min value = 0.3000205324689946
  2.618865 seconds (39.56 k allocations: 131.362 MiB, 0.41% gc time)
Possible interval YIZ: [0.3000205324689946, 0.3099167256054529]


### 8 qubits

In [54]:
function test_11_qubits(model, vars)
    #PAC framework:
    #Add in measurement data as constraints:
    @constraint(model, -0.9 ≤ vars["IIXIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIYIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["XIIIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["YIIIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["IIZIIIIX"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIIIXIIZ"] ≤ 0.81)
    @constraint(model, -0.4 ≤ vars["IIYIIIXI"] ≤ -0.38)
    @constraint(model, 0.3 ≤ vars["IYIZIIII"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IZIXIIII"] ≤ 0.25)
    @constraint(model, 0.3 ≤ vars["IIIIIYIZ"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IIIIIZIX"] ≤ 0.25)
    @constraint(model, 0.5 ≤ vars["IIIYZIII"] ≤ 0.55)
    @constraint(model, 0.5 ≤ vars["IIIIIYZI"] ≤ 0.55)
    @constraint(model, 0.72 ≤ vars["IIIIIXZI"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["YXIIIIII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIXZII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIYXII"] ≤ 0.74)
end

test_11_qubits (generic function with 1 method)

In [55]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 8 #8 qubits
d = 1 #level-1 pseudostate
k = 2 #looking at observables of at most order 2

(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k);


vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

test_11_qubits(model, vars)

@time res_interval = pseudostate_bound_expr(vars["IIXIIIII"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = -0.5346960295726031
  0.248197 seconds (63.75 k allocations: 38.414 MiB)
Possible interval: [-0.5346960295726031, 0.5346923598918945]
Time for second solve was 0.11071610450744629


In [63]:
function test_11_qubits_2(model, vars)
    #PAC framework:
    #Add in measurement data as constraints:
    @constraint(model, -0.9 ≤ vars["IIXIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIYIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["XIIIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["YIIIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["IIZIIIIX"] ≤ 0.6)
#     @constraint(model, 0.79 ≤ vars["IIIIXIIZ"] ≤ 0.81)
    @constraint(model, -0.4 ≤ vars["IIYIIIXI"] ≤ -0.38)
#     @constraint(model, 0.3 ≤ vars["IYIZIIII"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IZIXIIII"] ≤ 0.25)
#     @constraint(model, 0.3 ≤ vars["IIIIIYIZ"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IIIIIZIX"] ≤ 0.25)
#     @constraint(model, 0.5 ≤ vars["IIIYZIII"] ≤ 0.55)
    @constraint(model, 0.5 ≤ vars["IIIIIYZI"] ≤ 0.55)
#     @constraint(model, 0.72 ≤ vars["IIIIIXZI"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["YXIIIIII"] ≤ 0.74)
#     @constraint(model, 0.72 ≤ vars["IIIIXZII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIYXII"] ≤ 0.74)
end

test_11_qubits_2 (generic function with 1 method)

In [64]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 8 #8 qubits
d = 2 #level-2 pseudostate
k = 2 #looking at observables of at most order 2

(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k);


vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

test_11_qubits_2(model, vars)

@time res_interval = pseudostate_bound_expr(vars["IIXIIIII"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = -0.5347273453036312
 68.357837 seconds (2.79 M allocations: 3.120 GiB, 0.13% gc time)
Possible interval: [-0.5347273453036312, 0.5347282391754715]
Time for second solve was 37.02287316322327


## Pseudohistory

In [104]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 5 #5 qubits - minimum 'relevant' case since we lift from 1 up to 2 and back down.
d = 1 #level-1 pseudostate
k = 2 #looking at observables of at most order 2

2

In [115]:
model = JuMP.Model(optimizer_with_attributes(COSMO.Optimizer, "verbose" => false));

(row_ops_flat, row_III, vars) = make_qubit_SDP_vars(model, q, n, d, k);

make_pseudostate_model(model, vars, row_ops_flat, row_III)

#Make our initial estimates about the state
@constraint(model, 0.3 ≤ vars["IIIIX"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIIIY"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["IIIIZ"] ≤ -0.38)
@constraint(model, 0.3 ≤ vars["IIXXI"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIYYI"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["IIZZI"] ≤ -0.38)
;

In [116]:
ext_vars = extend_SDP_vars(model, d, vars);

In [117]:
model

A JuMP Model
Feasibility problem with:
Variables: 885
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 105 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 6 constraints
`Vector{AffExpr}`-in-`MathOptInterface.HermitianPositiveSemidefiniteConeTriangle`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 885 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 885 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COSMO
Names registered in the model: vars

In [118]:
#Step forward with a CNOT gate on qubits 3 and 4.
CNOT = Dict{String,Complex}("IIIII" => 1/2, "IIIXI" => 1/2, "IIZII" => 1/2, "IIZXI" => -1/2)
evolved_vars = project_SDP_vars(model, d, ext_vars, CNOT)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["XIIII", "YIIII", "ZIIII", "IXIII", "IYIII", "IZIII", "IIXII", "IIYII", "IIZII", "IIIXI"  …  "IIZIZ", "IIIXX", "IIIYX", "IIIZX", "IIIXY", "IIIYY", "IIIZY", "IIIXZ", "IIIYZ", "IIIZZ"]
And data, a 105-element Vector{VariableRef}:
 vars[XIIII]
 vars[YIIII]
 vars[ZIIII]
 vars[IXIII]
 vars[IYIII]
 vars[IZIII]
 vars[IIXII]
 vars[IIYII]
 vars[IIZII]
 vars[IIIXI]
 vars[IIIYI]
 vars[IIIZI]
 vars[IIIIX]
 ⋮
 vars[IIXIZ]
 vars[IIYIZ]
 vars[IIZIZ]
 vars[IIIXX]
 vars[IIIYX]
 vars[IIIZX]
 vars[IIIXY]
 vars[IIIYY]
 vars[IIIZY]
 vars[IIIXZ]
 vars[IIIYZ]
 vars[IIIZZ]

In [127]:
@time res_interval_1 = pseudostate_bound_expr(vars["IIIZI"])
@time res_interval_2 = pseudostate_bound_expr(evolved_vars["IIZZI"])
println(res_interval_1)
println(res_interval_2)

Min value = -0.6131014780678696
  4.829750 seconds (117.90 k allocations: 296.067 MiB, 0.30% gc time)
Min value = -0.6131055104741694
  4.714197 seconds (117.94 k allocations: 292.876 MiB, 0.19% gc time)
[-0.6131014780678696, 0.6130932583327002]
[-0.6131055104741694, 0.613100470169085]


In [128]:
@time res_interval_1 = pseudostate_bound_expr(vars["IIZZI"])
@time res_interval_2 = pseudostate_bound_expr(evolved_vars["IIIZI"])
println(res_interval_1)
println(res_interval_2)

Min value = -0.3999999987306233
  3.238750 seconds (116.05 k allocations: 232.273 MiB, 0.75% gc time)
Min value = -0.4000000000000147
  3.228444 seconds (116.10 k allocations: 230.685 MiB, 0.31% gc time)
[-0.3999999987306233, -0.37999999999999956]
[-0.4000000000000147, -0.380000000005707]
